# Deep Learning Approach with Tensorflow

In [2]:
import tensorflow as tf

Read in data

In [3]:
from utils import read_data
df, X, y1, y2 = read_data("~/GitHub/3pt-shooting/data/preprocessed.csv")
display(df.head())

,POS,AGE,TEAM,G,GS,MP,FG,FGA,FG%,3P,...,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,next_3P%,3pt_dif
0,PG,31.0,ATL,80,80,2195,268,644,0.416,126,...,3.0,1.7,4.8,0.104,0.1,-0.2,-0.2,1.0,0.433825,0.044825
1,SG,29.0,ATL,79,0,2460,496,1105,0.449,163,...,5.6,1.7,7.3,0.143,3.5,-1.1,2.4,2.7,0.341000,-0.041000
2,SF,31.0,ATL,79,5,1317,175,393,0.445,57,...,2.1,1.1,3.1,0.114,-0.4,-0.6,-0.9,0.4,0.326041,-0.010959
3,SG,28.0,ATL,76,76,2886,635,1386,0.458,129,...,6.0,2.4,8.4,0.140,3.4,-0.7,2.7,3.4,0.297000,-0.072000
4,SF,23.0,ATL,81,81,2468,302,664,0.455,40,...,3.1,2.6,5.7,0.112,-0.4,0.1,-0.4,1.0,0.336000,0.033000


Design keras model

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

def design_model(X):

    model = Sequential()
    model.add(InputLayer(input_shape=(X.shape[1],)))
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate = 0.01)
    model.compile(loss='mse', metrics=['mae'], optimizer=opt)
    return model